referring to this medium article:

https://wire.insiderfinance.io/statistical-arbitrage-in-python-how-ken-griffin-built-a-fortune-917758234152

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the basket of assets
tickers = ["AAPL", "MSFT", "GOOGL"]

# Download historical Adjusted Close prices (last 2 years)
data = yf.download(tickers, start="2023-01-01", end="2025-02-26")

# Check the data
print(data.head())

/var/folders/z_/9h44s4jd1_v60ytmp987cz8m0000gn/T/ipykernel_84149/3946870560.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2023-01-01", end="2025-02-26")
[*********************100%***********************]  3 of 3 completed

Price            Close                               High             \
Ticker            AAPL      GOOGL        MSFT        AAPL      GOOGL   
Date                                                                   
2023-01-03  123.470619  88.588707  234.808975  129.226060  90.507201   
2023-01-04  124.744125  87.554909  224.537674  127.014716  90.109587   
2023-01-05  123.421249  85.686111  217.882874  126.136083  87.047947   
2023-01-06  127.962440  86.819313  220.450668  128.623871  87.167233   
2023-01-09  128.485657  87.495255  222.597076  131.703978  89.513159   

Price                          Low                               Open  \
Ticker            MSFT        AAPL      GOOGL        MSFT        AAPL   
Date                                                                    
2023-01-03  240.856103  122.582127  87.992278  232.672380  128.613993   
2023-01-04  228.232587  123.480495  86.749732  221.460205  125.267347   
2023-01-05  223.018529  123.164580  85.387904  217.343823 

In [4]:
data.columns

MultiIndex([( 'Close',  'AAPL'),
            ( 'Close', 'GOOGL'),
            ( 'Close',  'MSFT'),
            (  'High',  'AAPL'),
            (  'High', 'GOOGL'),
            (  'High',  'MSFT'),
            (   'Low',  'AAPL'),
            (   'Low', 'GOOGL'),
            (   'Low',  'MSFT'),
            (  'Open',  'AAPL'),
            (  'Open', 'GOOGL'),
            (  'Open',  'MSFT'),
            ('Volume',  'AAPL'),
            ('Volume', 'GOOGL'),
            ('Volume',  'MSFT')],
           names=['Price', 'Ticker'])

In [7]:
close_data = data['Close']

In [9]:
close_data.min()

Ticker
AAPL     123.421249
GOOGL     85.686111
MSFT     217.882874
dtype: float64